In [ ]:
import sys

# pandas读写csv文件
import pandas as pd
import numpy as np

path = '/Users/mingjue/downloads/'

df_train_1 = pd.read_csv(path + 'data_format1/train_format1.csv', dtype={'user_id': 'str', 'merchant_id': 'str', 'label': 'float'})
df_test_1 = pd.read_csv(path + 'data_format1/test_format1.csv', dtype={'user_id': 'str', 'merchant_id': 'str', 'label': 'float'})
df_user_info = pd.read_csv(path + 'data_format1/user_info_format1.csv', dtype='str')
df_user_log = pd.read_csv(path + 'data_format1/user_log_format1.csv', dtype='str')
df_train_2 = pd.read_csv(path + 'data_format2/train_format2.csv', dtype='str')
df_test_2 = pd.read_csv(path + 'data_format2/test_format2.csv', dtype='str')
df_infer = pd.read_csv(path + 'sample_submission.csv', dtype='str')

# df_user_info
# age_range 和 gender 有缺失值
# age_range 的取值为0-8，0表示缺失值（age_range一般对应年龄大小顺序，根据分布判断0表示未知）
# gender 的取值为0-2，2表示缺失值(根据官网描述，2和NULL表示未知)
# 将df_user_info的 age_range 列  字段转换为float， 取值0都替换为空值,
df_user_info['age_range'] = df_user_info['age_range'].astype(float)
df_user_info['age_range'] = df_user_info['age_range'].replace(0, np.nan)
# 将df_user_info的 gender 列 中 取值为2替换为空值
df_user_info['gender'] = df_user_info['gender'].replace('2', np.nan)

# df_user_log
df_user_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)
# time_stamp 的取值为0511到1112共186天，表示mmdd，没有年份，请转换为标准日期格式
df_user_log['time_stamp'] = pd.to_datetime('2025' + df_user_log['time_stamp'], format='%Y%m%d')
df_user_log['recency'] = (pd.to_datetime('2025-11-12') - df_user_log['time_stamp']).dt.days + 1

# 计算用户rfm, 数据没有monetary，所以不能计算rfm，只能计算rf
# recency：距离最后一次消费的天数
# frequency：消费的次数

df_user_rf = df_user_log.groupby('user_id').agg(recency=('recency', 'min'), frequency=('user_id', 'count'),).reset_index()
df_user_rf_0 = df_user_log.query('"0"==action_type').groupby('user_id').agg(recency_0=('recency', 'min'), frequency_0=('user_id', 'count'),).reset_index()
df_user_rf_1 = df_user_log.query('"1"==action_type').groupby('user_id').agg(recency_1=('recency', 'min'), frequency_1=('user_id', 'count'),).reset_index()
df_user_rf_2 = df_user_log.query('"2"==action_type').groupby('user_id').agg(recency_2=('recency', 'min'), frequency_2=('user_id', 'count'),).reset_index()
df_user_rf_3 = df_user_log.query('"3"==action_type').groupby('user_id').agg(recency_3=('recency', 'min'), frequency_3=('user_id', 'count'),).reset_index()
df_user_rf = df_user_rf.merge(
    df_user_rf_0, how='left', on='user_id',
).merge(
    df_user_rf_1, how='left', on='user_id',
).merge(
    df_user_rf_2, how='left', on='user_id',
).merge(
    df_user_rf_3, how='left', on='user_id',
)
del df_user_rf_0, df_user_rf_1, df_user_rf_2, df_user_rf_3,

# 计算 user_id, merchant_id 交互特征
df_user_merchant_rf = df_user_log.groupby(['user_id', 'merchant_id']).agg(recency_um=('recency', 'min'),frequency_um=('user_id', 'count'),).reset_index()
df_user_merchant_rf_0 = df_user_log.query('"0"==action_type').groupby(['user_id', 'merchant_id']).agg(recency_um_0=('recency', 'min'),frequency_um_0=('user_id', 'count'),).reset_index()
df_user_merchant_rf_1 = df_user_log.query('"1"==action_type').groupby(['user_id', 'merchant_id']).agg(recency_um_1=('recency', 'min'),frequency_um_1=('user_id', 'count'),).reset_index()
df_user_merchant_rf_2 = df_user_log.query('"2"==action_type').groupby(['user_id', 'merchant_id']).agg(recency_um_2=('recency', 'min'),frequency_um_2=('user_id', 'count'),).reset_index()
df_user_merchant_rf_3 = df_user_log.query('"3"==action_type').groupby(['user_id', 'merchant_id']).agg(recency_um_3=('recency', 'min'),frequency_um_3=('user_id', 'count'),).reset_index()
df_user_merchant_rf = df_user_merchant_rf.merge(
    df_user_merchant_rf_0, how='left', on=['user_id', 'merchant_id'],
).merge(
    df_user_merchant_rf_1, how='left', on=['user_id', 'merchant_id'],
).merge(
    df_user_merchant_rf_2, how='left', on=['user_id', 'merchant_id'],
).merge(
    df_user_merchant_rf_3, how='left', on=['user_id', 'merchant_id'],
)
del df_user_merchant_rf_0, df_user_merchant_rf_1, df_user_merchant_rf_2, df_user_merchant_rf_3,

In [10]:
# 计算 merchant_id 特征
df_merchant = df_user_log.groupby('merchant_id').agg(recency_m=('recency', 'min'),frequency_m=('user_id', 'count'),).reset_index()
df_merchant_0 = df_user_log.query('"0"==action_type').groupby('merchant_id').agg(recency_m_0=('recency', 'min'),frequency_m_0=('user_id', 'count'),).reset_index()
df_merchant_1 = df_user_log.query('"1"==action_type').groupby('merchant_id').agg(recency_m_1=('recency', 'min'),frequency_m_1=('user_id', 'count'),).reset_index()
df_merchant_2 = df_user_log.query('"2"==action_type').groupby('merchant_id').agg(recency_m_2=('recency', 'min'),frequency_m_2=('user_id', 'count'),).reset_index()
df_merchant_3 = df_user_log.query('"3"==action_type').groupby('merchant_id').agg(recency_m_3=('recency', 'min'),frequency_m_3=('user_id', 'count'),).reset_index()
df_merchant = pd.merge(df_merchant, df_merchant_0, on='merchant_id', how='left')
df_merchant = pd.merge(df_merchant, df_merchant_1, on='merchant_id', how='left')
df_merchant = pd.merge(df_merchant, df_merchant_2, on='merchant_id', how='left')
df_merchant = pd.merge(df_merchant, df_merchant_3, on='merchant_id', how='left')
del df_merchant_0, df_merchant_1, df_merchant_2, df_merchant_3,



In [ ]:
# df_infer = pd.read_csv(path + 'sample_submission.csv')
# # 简单的预测，随机大于0小于1的小数，比如0.5、0.8，均值0.2
# df_infer['prob'] = np.random.rand(len(df_infer))
# # 保存结果到prediction.csv文件中
# df_infer.to_csv(path + 'prediction.csv', index=False)

In [12]:
df_train = df_train_1.merge(
    df_user_info, on='user_id', how='left',
).merge(
    df_user_merchant_rf, on=['user_id', 'merchant_id'], how='left',
).merge(
    df_user_rf, on='user_id', how='left',
).merge(
    df_merchant, on='merchant_id', how='left',
)
df_train['gender'] = df_train['gender'].astype(str)

df_test = df_test_1.merge(
    df_user_info, on='user_id', how='left',
).merge(
    df_user_merchant_rf, on=['user_id', 'merchant_id'], how='left',
).merge(
    df_user_rf, on='user_id', how='left',
).merge(
    df_merchant, on='merchant_id', how='left',
)
df_test['gender'] = df_test['gender'].astype(str)

In [13]:
# df_train catboost 二分类 训练推理
# 固定随机种子，保证结果可复现
np.random.seed(42)
# 训练代码 
from catboost import CatBoostClassifier, Pool  
from sklearn.model_selection import train_test_split
features = [
    'age_range', 'gender',
    'recency_um', 'frequency_um',
    'recency_um_0', 'frequency_um_0',
    'recency_um_1', 'frequency_um_1',
    'recency_um_2', 'frequency_um_2',
    'recency_um_3', 'frequency_um_3',
    'recency', 'frequency',
    'recency_0', 'frequency_0',
    'recency_1', 'frequency_1',
    'recency_2', 'frequency_2',
    'recency_3', 'frequency_3',
    'recency_m', 'frequency_m',
    'recency_m_0', 'frequency_m_0',
    'recency_m_1', 'frequency_m_1',
    'recency_m_2', 'frequency_m_2',
    'recency_m_3', 'frequency_m_3',
]
target = 'label'
X = df_train[features]
y = df_train[target].astype(float)
X_test = df_test[features]        
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
train_pool = Pool(X_train, y_train, cat_features=['gender'])
val_pool = Pool(X_val, y_val, cat_features=['gender'])  
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, eval_metric='AUC', random_seed=42, logging_level='Verbose', use_best_model=True)
model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=100)
y_pred = model.predict_proba(X_test)[:, 1]
df_test['prob'] = y_pred
df_test[['user_id', 'merchant_id', 'prob']].to_csv(path + 'prediction.csv', index=False)  
print(path + 'prediction.csv saved!')


0:	test: 0.5550449	best: 0.5550449 (0)	total: 115ms	remaining: 1m 54s
100:	test: 0.6548404	best: 0.6548404 (100)	total: 4.76s	remaining: 42.4s
200:	test: 0.6671818	best: 0.6671818 (200)	total: 9.35s	remaining: 37.2s
300:	test: 0.6693877	best: 0.6694621 (294)	total: 13.9s	remaining: 32.3s
400:	test: 0.6700049	best: 0.6704812 (365)	total: 18.5s	remaining: 27.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.670481154
bestIteration = 365

Shrink model to first 366 iterations.
/Users/mingjue/downloads/prediction.csv saved!
